# 데이터 프로파일링 진행 (Spark 사용)

### 문제 해결 주제
- 71페이지까지의 일반적인 의문 + 내가 궁금한 것들까지 추가하여 진행해보기

In [12]:
spark.stop()

In [13]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("firecall").getOrCreate()

In [14]:
df = spark.read.format('csv').load('sf-fire-calls.csv', inferSchema=True, header=True)

In [15]:
df

DataFrame[CallNumber: int, UnitID: string, IncidentNumber: int, CallType: string, CallDate: string, WatchDate: string, CallFinalDisposition: string, AvailableDtTm: string, Address: string, City: string, Zipcode: int, Battalion: string, StationArea: string, Box: string, OriginalPriority: string, Priority: string, FinalPriority: int, ALSUnit: boolean, CallTypeGroup: string, NumAlarms: int, UnitType: string, UnitSequenceInCallDispatch: int, FirePreventionDistrict: string, SupervisorDistrict: string, Neighborhood: string, Location: string, RowID: string, Delay: double]

In [16]:
df.printSchema() #컬럼 확인

root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: integer (nullable = true)
 |-- ALSUnit: boolean (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: integer (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: integer (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 

In [17]:
df.select('CallType').distinct().count() #신고타입의 종류수

30

In [20]:
df.createOrReplaceTempView("fireDF") # 데이터프레임을 SQL 테이블로 등록
spark.sql("""
    SELECT COUNT(DISTINCT CallType) AS DistinctCallTypes
    FROM fireDF
""").show() #신고타입의 종류수2

+-----------------+
|DistinctCallTypes|
+-----------------+
|               30|
+-----------------+



In [21]:
# 컬럼 이름 변경 및 특정 조건에 따라 출력
df = df.withColumnRenamed('Delay', 'ResponseDelayedMins')
df.createOrReplaceTempView("fireDF") # SQL 테이블 등록
spark.sql("""
    SELECT CallNumber, IncidentNumber, ResponseDelayedMins
    FROM fireDF
    WHERE ResponseDelayedMins >= 5
""").show()

+----------+--------------+-------------------+
|CallNumber|IncidentNumber|ResponseDelayedMins|
+----------+--------------+-------------------+
|  20110315|       2003409|               5.35|
|  20120147|       2003642|               6.25|
|  20130013|       2003818|                5.2|
|  20140067|       2004152|                5.6|
|  20140177|       2004216|               7.25|
|  20150056|       2004408|          11.916667|
|  20150254|       2004521|           5.116667|
|  20150265|       2004528|           8.633333|
|  20150265|       2004528|           95.28333|
|  20150380|       2004610|               5.45|
|  20150414|       2004641|                7.6|
|  20160059|       2004721|           6.133333|
|  20160064|       2004724|          5.1833334|
|  20170118|       2005038|          6.9166665|
|  20170342|       2005173|                5.2|
|  20180129|       2005321|               6.35|
|  20180191|       2005353|           7.983333|
|  20180382|       2005480|             

# 여기부턴 데이터 내용 분석

In [24]:
# 신고 타입별 신고 건수
from pyspark.sql.functions import count

df.rollup('CallType').agg(count('*').alias('CallCount')).orderBy('CallCount', ascending=False).show()

+--------------------+---------+
|            CallType|CallCount|
+--------------------+---------+
|                null|   175296|
|    Medical Incident|   113794|
|      Structure Fire|    23319|
|              Alarms|    19406|
|   Traffic Collision|     7013|
|Citizen Assist / ...|     2524|
|               Other|     2166|
|        Outside Fire|     2094|
|        Vehicle Fire|      854|
|Gas Leak (Natural...|      764|
|        Water Rescue|      755|
|Odor (Strange / U...|      490|
|   Electrical Hazard|      482|
|Elevator / Escala...|      453|
|Smoke Investigati...|      391|
|          Fuel Spill|      193|
|              HazMat|      124|
|Industrial Accidents|       94|
|           Explosion|       89|
|Train / Rail Inci...|       57|
+--------------------+---------+
only showing top 20 rows



In [53]:
# 가장 흔한 형태의 신고를 포함한 Rollup 형태 결과(SQL문으로)
spark.sql("""
    SELECT 
        CallType,
        COUNT(*) AS CallCount
    FROM fireDF
    GROUP BY ROLLUP(CallType)
    ORDER BY CallCount DESC
""").show()

+--------------------+---------+
|            CallType|CallCount|
+--------------------+---------+
|                null|   175296|
|    Medical Incident|   113794|
|      Structure Fire|    23319|
|              Alarms|    19406|
|   Traffic Collision|     7013|
|Citizen Assist / ...|     2524|
|               Other|     2166|
|        Outside Fire|     2094|
|        Vehicle Fire|      854|
|Gas Leak (Natural...|      764|
|        Water Rescue|      755|
|Odor (Strange / U...|      490|
|   Electrical Hazard|      482|
|Elevator / Escala...|      453|
|Smoke Investigati...|      391|
|          Fuel Spill|      193|
|              HazMat|      124|
|Industrial Accidents|       94|
|           Explosion|       89|
|Train / Rail Inci...|       57|
+--------------------+---------+
only showing top 20 rows



### 신고 타입별 분석
- 의료 사건(Medical Incident)이 가장 많았고, 화재 관련(Structure Fire) 신고가 그 뒤를 이음.

In [54]:
# 응답 대기 시간의 평균
from pyspark.sql.functions import avg
df.select(avg('ResponseDelayedMins').alias('AverageResponseDelay')).show()

+--------------------+
|AverageResponseDelay|
+--------------------+
|  3.8923641541750342|
+--------------------+



In [52]:
# 신고 유형별 처리 시간 분석(각 타입에 따라 처리되는 시간값 구하기)
df.groupBy("CallType").agg(avg("ResponseDelayedMins").alias("AvgResponseTime")).orderBy("AvgResponseTime", ascending=False).show(100)

+--------------------+------------------+
|            CallType|   AvgResponseTime|
+--------------------+------------------+
|Mutual Aid / Assi...| 38.41666631111111|
|       Assist Police|26.981903994285716|
|Train / Rail Inci...|16.452046763157895|
|      Administrative|12.261111333333332|
|              HazMat| 7.527016126612902|
|         Marine Fire| 6.928571314285715|
|Confined Space / ...| 6.915384576923078|
|Watercraft in Dis...| 6.886904817857142|
|  Suspicious Package|6.5766667199999995|
|   High Angle Rescue|6.0489583750000016|
|        Water Rescue| 5.507748342145695|
|               Other| 5.505155432421978|
|          Fuel Spill| 5.492227982383421|
|Citizen Assist / ...| 5.473342576604596|
|   Electrical Hazard| 5.178112038174274|
|Industrial Accidents| 5.014716334042553|
|           Oil Spill| 4.977777761904762|
|Odor (Strange / U...| 4.947959182000003|
|Gas Leak (Natural...|  4.58339877840314|
|Smoke Investigati...| 4.466069897851662|
|Extrication / Ent...| 4.391666678

### 신고유형별 분석
- 전체 평균 대비 높은 대기 시간이 소량의 일부 유형에서 발생, 자원 배분 효율화 필요.
- **Medical Incident**는 현재도 대기 시간이 짧으나 신고 비중이 높은 만큼, 현 상태를 유지하기 위한 지속적 자원 확보 필요.

# 시간대별 분석

In [27]:
# 신고가 가장 많은 달은? 10월
from pyspark.sql.functions import month, count, to_timestamp

#달만 출력하여 진행
df.withColumn("IncidentDate", to_timestamp("CallDate", "MM/dd/yyyy")) \
    .withColumn("Month", month("IncidentDate")) \
    .groupBy("Month").agg(count("*").alias("CallCount")).orderBy("CallCount", ascending=False).show() 

+-----+---------+
|Month|CallCount|
+-----+---------+
|   10|    15410|
|    8|    15126|
|    5|    15099|
|    9|    14991|
|   12|    14782|
|    7|    14762|
|    1|    14586|
|    3|    14582|
|    6|    14553|
|    4|    14140|
|   11|    13863|
|    2|    13402|
+-----+---------+



In [42]:
# 의료사고가 가장 많은 달은? 5월

df.filter(df["CallType"] == "Medical Incident").withColumn("IncidentDate", to_timestamp("CallDate", "MM/dd/yyyy")) \
    .withColumn("Month", month("IncidentDate")) \
    .groupBy("Month").agg(count("*").alias("CallCount")).orderBy("CallCount", ascending=False).show() 

+-----+---------+
|Month|CallCount|
+-----+---------+
|    5|     9962|
|   10|     9937|
|    8|     9869|
|    9|     9772|
|    3|     9659|
|   12|     9474|
|    1|     9469|
|    6|     9434|
|    7|     9320|
|    4|     9181|
|   11|     8860|
|    2|     8857|
+-----+---------+



### 의료사고가 가장 많은 달 분석
-  5월과 10월이 의료사고 신고가 집중되는 경향을 보이나 큰 차이는 없음, 2월엔 상대적으로 적은 의료사고량 추정

In [43]:
# 화재사고가 가장 많은 달은? 7월

df.filter(df["CallType"] == "Structure Fire").withColumn("IncidentDate", to_timestamp("CallDate", "MM/dd/yyyy")) \
    .withColumn("Month", month("IncidentDate")) \
    .groupBy("Month").agg(count("*").alias("CallCount")).orderBy("CallCount", ascending=False).show() 

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
+-----+---------+
|Month|CallCount|
+-----+---------+
|    7|     2116|
|    6|     2053|
|   10|     2027|
|    5|     2017|
|    9|     1996|
|   12|     1946|
|    8|     1920|
|    1|     1916|
|    4|     1909|
|   11|     1898|
|    3|     1821|
|    2|     1700|
+-----+---------+



### 화재 사고가 가장 많은 달 분석
- 여름철(6월~7월)에 화재사고 신고가 급증하는 경향을 보임.여름철에 구조 관련 장비 및 인력을 증대하여 효율적인 대처를 준비.

In [37]:
# 연도별 신고 추세
from pyspark.sql.functions import year

df.withColumn("IncidentDate", to_timestamp("CallDate", "MM/dd/yyyy")).withColumn("Year", year("IncidentDate")) \
    .groupBy("Year").agg(count("*").alias("CallCount")).orderBy("Year").show()

+----+---------+
|Year|CallCount|
+----+---------+
|2000|     5459|
|2001|     7713|
|2002|     8090|
|2003|     8499|
|2004|     8283|
|2005|     8282|
|2006|     8174|
|2007|     8255|
|2008|     8869|
|2009|     8789|
|2010|     9341|
|2011|     9735|
|2012|     9674|
|2013|    10020|
|2014|    10775|
|2015|    11458|
|2016|    11609|
|2017|    12135|
|2018|    10136|
+----+---------+



### 연도별 신고 추세 분석
- 2000년부터 2017년까지 신고 건수가 꾸준히 증가하는 추세를 보임, 증가하는 신고 수요에 대비해 지속적인 인프라 확충

In [55]:
# 시간대별 신고추세, 사람들이 주로 활동하는 시간에 많은 신고량
from pyspark.sql.functions import hour

df.withColumn("AvailableDtTS", to_timestamp("AvailableDtTm", "MM/dd/yyyy hh:mm:ss a")).withColumn("Hour", hour("AvailableDtTS")) \
    .groupBy("Hour").agg(count("*").alias("CallCount")).orderBy("Hour").show(25)

+----+---------+
|Hour|CallCount|
+----+---------+
|null|     1794|
|   0|     5851|
|   1|     5418|
|   2|     5144|
|   3|     4291|
|   4|     3573|
|   5|     3265|
|   6|     3455|
|   7|     4589|
|   8|     5930|
|   9|     7325|
|  10|     8419|
|  11|     8989|
|  12|     9476|
|  13|     9515|
|  14|     9535|
|  15|     9538|
|  16|     9574|
|  17|     9720|
|  18|     9637|
|  19|     9375|
|  20|     8814|
|  21|     8009|
|  22|     7440|
|  23|     6620|
+----+---------+



# 지역별 분석

In [45]:
# 가장 신고가 많은 지역은? Tenderloin
df.groupBy("Neighborhood").agg(count("*").alias("CallCount")).orderBy("CallCount", ascending=False).show()

+--------------------+---------+
|        Neighborhood|CallCount|
+--------------------+---------+
|          Tenderloin|    22785|
|     South of Market|    16623|
|             Mission|    15912|
|Financial Distric...|    11912|
|Bayview Hunters P...|     9616|
|     Sunset/Parkside|     6984|
|    Western Addition|     6591|
|            Nob Hill|     5810|
|      Outer Richmond|     4604|
|        Hayes Valley|     4357|
| Castro/Upper Market|     4176|
|  West of Twin Peaks|     4005|
|         North Beach|     3800|
|           Chinatown|     3731|
|           Excelsior|     3643|
|     Pacific Heights|     3622|
|              Marina|     3415|
|        Potrero Hill|     3321|
|      Bernal Heights|     3260|
|        Inner Sunset|     2959|
+--------------------+---------+
only showing top 20 rows



In [46]:
# 화재사고가 가장 많은 지역은?
df.filter(df["CallType"] == "Structure Fire").groupBy("Neighborhood").agg(count("*").alias("CallCount")).orderBy("CallCount", ascending=False).show()

+--------------------+---------+
|        Neighborhood|CallCount|
+--------------------+---------+
|          Tenderloin|     2385|
|             Mission|     2348|
|     South of Market|     1666|
|Bayview Hunters P...|     1613|
|Financial Distric...|     1242|
|            Nob Hill|      938|
|    Western Addition|      820|
|     Sunset/Parkside|      798|
| Castro/Upper Market|      686|
|        Hayes Valley|      649|
|           Excelsior|      628|
|              Marina|      618|
|           Chinatown|      598|
|      Outer Richmond|      589|
|     Pacific Heights|      586|
|  West of Twin Peaks|      526|
|        Potrero Hill|      507|
|      Bernal Heights|      504|
|        Russian Hill|      487|
|      Haight Ashbury|      468|
+--------------------+---------+
only showing top 20 rows



### 지역별 화재 사고 비율
- Tenderloin 지역은 **전체 신고**와 **화재사고** 모두에서 가장 높은 신고 건수를 기록. 인구가 가장 많을 수도 있어보임. 충분한 인프라 확보 필요
- 전체 신고량 대비 화재사고의 비중이 높은 지역은 **Mission**과 **Bayview Hunters Point**, 건조하거나 상업 지역일 확률 높아보이며 집중 관찰

In [47]:
# 의료 사고가 가장 많았던 지역은? 
df.filter(df["CallType"] == "Medical Incident").groupBy("Neighborhood").agg(count("*").alias("CallCount")).orderBy("CallCount", ascending=False).show()

+--------------------+---------+
|        Neighborhood|CallCount|
+--------------------+---------+
|          Tenderloin|    17109|
|     South of Market|    12181|
|             Mission|    10796|
|Financial Distric...|     6532|
|Bayview Hunters P...|     6099|
|     Sunset/Parkside|     4936|
|    Western Addition|     4191|
|            Nob Hill|     3516|
|      Outer Richmond|     3075|
|        Hayes Valley|     2731|
| Castro/Upper Market|     2666|
|  West of Twin Peaks|     2545|
|           Excelsior|     2535|
|         North Beach|     2377|
|           Chinatown|     2215|
|      Bernal Heights|     2096|
|        Potrero Hill|     1956|
|     Pacific Heights|     1913|
|              Marina|     1835|
|        Inner Sunset|     1800|
+--------------------+---------+
only showing top 20 rows



### 지역별 의료 사고 비율
- **Tenderloin**과 **South of Market** 지역은 전체 신고량 중 의료 사고의 비중이 가장 높아, 해당 지역은 의료 관련 응급 서비스 수요가 매우 클듯.
- **Financial District/South Beach**는 의료 사고 비중이 비교적 낮으나 여전히 주요 신고 유형으로 확인됨. 이는 상업지역의 특성으로 보임. 

In [50]:
# 어떤 주에서 가장 신고가 많았는가? SF
df.groupBy("City").agg(count("*").alias("CallCount")).orderBy("CallCount", ascending=False).show()

+---------------+---------+
|           City|CallCount|
+---------------+---------+
|             SF|   120072|
|  San Francisco|    51739|
|  SAN FRANCISCO|     1676|
|             TI|      486|
|       Presidio|      281|
|             PR|      257|
|  Treasure Isla|      228|
|           null|      207|
|    Yerba Buena|       41|
|             DC|       41|
|            SFO|       37|
|     Fort Mason|       36|
|             FM|       36|
|             YB|       34|
|             HP|       31|
|  Hunters Point|       27|
|       PRESIDIO|       13|
|Treasure Island|       12|
|            OAK|       11|
|             BN|        9|
+---------------+---------+
only showing top 20 rows



In [56]:
spark.stop()